In [ ]:
# WIP Initial simple RAG example with eval

In [1]:
!pip install -qqq -U scikit-learn

In [2]:
import json
import pathlib
from datetime import datetime
from typing import List

import wandb



In [3]:
WANDB_ENTITY = "parambharat"
WANDB_PROJECT = "advanced_rag"

wandb.require("core")

run = wandb.init(entity=WANDB_ENTITY, project=WANDB_PROJECT, job_type="data_ingestion", group="initial_example")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: parambharat. Use `wandb login --relogin` to force relogin


## Data ingestion

### Loading the data

In [4]:
docs_dir = pathlib.Path("../data/wandb_docs_06_24")
docs_files = sorted(docs_dir.rglob("*.md"))

print(f"Number of files: {len(docs_files)}\n")
print("First 5 files:\n{files}".format(files='\n'.join(map(str, docs_files[:5]))))

Number of files: 380

First 5 files:
../data/wandb_docs_06_24/guides/app/features/anon.md
../data/wandb_docs_06_24/guides/app/features/custom-charts/intro.md
../data/wandb_docs_06_24/guides/app/features/custom-charts/walkthrough.md
../data/wandb_docs_06_24/guides/app/features/intro.md
../data/wandb_docs_06_24/guides/app/features/notes.md


In [5]:
from IPython.display import Markdown
print(docs_files[0].read_text())

---
description: Log and visualize data without a W&B account
displayed_sidebar: default
---

# Anonymous Mode

Are you publishing code that you want anyone to be able to run easily? Use Anonymous Mode to let someone run your code, see a W&B dashboard, and visualize results without needing to create a W&B account first.

Allow results to be logged in Anonymous Mode with `wandb.init(`**`anonymous="allow"`**`)`

:::info
**Publishing a paper?** Please [cite W&B](https://docs.wandb.ai/company/academics#bibtex-citation), and if you have questions about how to make your code accessible while using W&B, reach out to us at support@wandb.com.
:::

### How does someone without an account see results?

If someone runs your script and you have to set `anonymous="allow"`:

1. **Auto-create temporary account:** W&B checks for an account that's already signed in. If there's no account, we automatically create a new anonymous account and save that API key for the session.
2. **Log results quickly:** T

In [6]:
data = []
for file in docs_files:
    content = file.read_text()
    data.append({
        "content": content,
        "metadata": {
            "source": str(file.relative_to(docs_dir)),
            "raw_tokens": len(content.split())
        }})
data[:2]

[{'content': '---\ndescription: Log and visualize data without a W&B account\ndisplayed_sidebar: default\n---\n\n# Anonymous Mode\n\nAre you publishing code that you want anyone to be able to run easily? Use Anonymous Mode to let someone run your code, see a W&B dashboard, and visualize results without needing to create a W&B account first.\n\nAllow results to be logged in Anonymous Mode with `wandb.init(`**`anonymous="allow"`**`)`\n\n:::info\n**Publishing a paper?** Please [cite W&B](https://docs.wandb.ai/company/academics#bibtex-citation), and if you have questions about how to make your code accessible while using W&B, reach out to us at support@wandb.com.\n:::\n\n### How does someone without an account see results?\n\nIf someone runs your script and you have to set `anonymous="allow"`:\n\n1. **Auto-create temporary account:** W&B checks for an account that\'s already signed in. If there\'s no account, we automatically create a new anonymous account and save that API key for the ses

In [7]:
total_tokens = sum(map(lambda x: x["metadata"]["raw_tokens"], data))
print(f"Total Tokens in dataset: {total_tokens}")

Total Tokens in dataset: 246998


### Chunking the data

In [8]:
from typing import List

CHUNK_SIZE = 500
CHUNK_OVERLAP = 0

def split_into_chunks(text: str, chunk_size: int = CHUNK_SIZE, chunk_overlap: int = CHUNK_OVERLAP) -> List[str]:
    """Function to split the text into chunks of a maximum number of tokens
    ensure that the chunks are of size CHUNK_SIZE and overlap by chunk_overlap tokens
    use the `tokenizer.encode` method to tokenize the text
    """
    tokens = text.split()
    chunks = []
    start = 0
    while start < len(tokens):
        end = start + chunk_size
        chunk = tokens[start:end]
        chunks.append(" ".join(chunk))
        start = end - chunk_overlap
    return chunks


In [9]:
raw_artifact = run.use_artifact(f'{WANDB_ENTITY}/{WANDB_PROJECT}/raw_data:latest', type='dataset')
artifact_dir = raw_artifact.download()
raw_data_file = pathlib.Path(f"{artifact_dir}/documents.jsonl")
raw_data = list(map(json.loads, raw_data_file.read_text().splitlines()))
raw_data[:2]

2024/06/28 13:16:48 [DEBUG] GET https://storage.googleapis.com/wandb-production.appspot.com/parambharat/advanced_rag/aaaobcp1/artifact/930835866/wandb_manifest.json?Expires=1719564408&GoogleAccessId=gorilla-files-url-signer-man%40wandb-production.iam.gserviceaccount.com&Signature=t7uZxuIuFA5wUgnQxn2nFv6i4YD5RIZliKUw0tGIZbso4XqB0DzL7w1bn7Leue%2BRyDrpGOpbo6ZuSEZv%2BWkxaJRDYN%2Bjw0eqR4XVRzpM7Rh16SwGhOBX0T0lHcp1qQi%2BiIYdj3Nv6uI3cvgWb6r6oKAkMl6Z02NLev3d6%2FQsggKxp0%2FWqokWD0N1w7TINkOsnOPeT70nMebATbWpoawO9ScF7yfwAw%2BHXFtjIC21ZqdBx6tWPX9Xg3CXvpRV%2BK3y5XjP%2Bz4wXhSw8ilFHpoPlgcPHMKNGPgFkm%2BQvdY1Ww8ICJIwcBJ7JEjOOp6l0AIQwG%2FSO4tr%2BJXlKIubWupx0A%3D%3D


[{'content': '---\ndescription: Log and visualize data without a W&B account\ndisplayed_sidebar: default\n---\n\n# Anonymous Mode\n\nAre you publishing code that you want anyone to be able to run easily? Use Anonymous Mode to let someone run your code, see a W&B dashboard, and visualize results without needing to create a W&B account first.\n\nAllow results to be logged in Anonymous Mode with `wandb.init(`**`anonymous="allow"`**`)`\n\n:::info\n**Publishing a paper?** Please [cite W&B](https://docs.wandb.ai/company/academics#bibtex-citation), and if you have questions about how to make your code accessible while using W&B, reach out to us at support@wandb.com.\n:::\n\n### How does someone without an account see results?\n\nIf someone runs your script and you have to set `anonymous="allow"`:\n\n1. **Auto-create temporary account:** W&B checks for an account that\'s already signed in. If there\'s no account, we automatically create a new anonymous account and save that API key for the ses

In [10]:
chunked_data = []
for doc in raw_data:
    chunks = split_into_chunks(doc["content"])
    for chunk in chunks:
        chunked_data.append(
            {
                "content" : chunk,
                "metadata": {
                    "source": doc["metadata"]["source"],
                    "raw_tokens": len(chunk.split())
            }})

### Cleaning the data

In [11]:
def make_text_tokenization_safe(content: str) -> str:
    
    special_tokens_set = {'<|endofprompt|>', '<|endoftext|>', '<|fim_middle|>', '<|fim_prefix|>', '<|fim_suffix|>'}

    def remove_special_tokens(text: str) -> str:
        """Removes special tokens from the given text.

        Args:
            text: A string representing the text.

        Returns:
            The text with special tokens removed.
        """
        for token in special_tokens_set:
            text = text.replace(token, "")
        return text

    cleaned_content = remove_special_tokens(content)
    return cleaned_content

In [13]:
cleaned_data=[]
for doc in chunked_data:
    cleaned_doc = doc.copy()
    cleaned_doc["cleaned_content"] = make_text_tokenization_safe(doc["content"])
    cleaned_doc["metadata"]["cleaned_tokens"] = len(cleaned_doc["cleaned_content"].split())
    cleaned_data.append(cleaned_doc)
cleaned_data[:2]

[{'content': '--- description: Log and visualize data without a W&B account displayed_sidebar: default --- # Anonymous Mode Are you publishing code that you want anyone to be able to run easily? Use Anonymous Mode to let someone run your code, see a W&B dashboard, and visualize results without needing to create a W&B account first. Allow results to be logged in Anonymous Mode with `wandb.init(`**`anonymous="allow"`**`)` :::info **Publishing a paper?** Please [cite W&B](https://docs.wandb.ai/company/academics#bibtex-citation), and if you have questions about how to make your code accessible while using W&B, reach out to us at support@wandb.com. ::: ### How does someone without an account see results? If someone runs your script and you have to set `anonymous="allow"`: 1. **Auto-create temporary account:** W&B checks for an account that\'s already signed in. If there\'s no account, we automatically create a new anonymous account and save that API key for the session. 2. **Log results qui

In [14]:
total_raw_tokens = sum(map(lambda x: x["metadata"]["raw_tokens"], cleaned_data))
total_cleaned_tokens = sum(map(lambda x: x["metadata"]["cleaned_tokens"], cleaned_data))

chunked_artifact = wandb.Artifact(name="chunked_data", type="dataset",
description="Chunked wandb documentation", metadata={
    "total_files": len(cleaned_data),
    "date_processed": datetime.now().strftime("%Y-%m-%d"),
    "total_raw_tokens": total_raw_tokens,
    "total_cleaned_tokens": total_cleaned_tokens, 
    "chunk_size": CHUNK_SIZE,
    "chunk_overlap": CHUNK_OVERLAP
    }
)
with chunked_artifact.new_file("documents.jsonl", mode="w") as f:
    for item in cleaned_data:
        f.write(json.dumps(item) + "\n")
run.log_artifact(chunked_artifact)

<Artifact chunked_data>

## Vectorizing the data

In [16]:
chunked_artifact = run.use_artifact(f'{WANDB_ENTITY}/{WANDB_PROJECT}/chunked_data:latest', type='dataset')
artifact_dir = chunked_artifact.download()
chunked_data_file = pathlib.Path(f"{artifact_dir}/documents.jsonl")
chunked_data = list(map(json.loads, chunked_data_file.read_text().splitlines()))
chunked_data[:2]

2024/06/28 13:18:00 [DEBUG] GET https://storage.googleapis.com/wandb-production.appspot.com/parambharat/advanced_rag/ub0tl935/artifact/932078705/wandb_manifest.json?Expires=1719564480&GoogleAccessId=gorilla-files-url-signer-man%40wandb-production.iam.gserviceaccount.com&Signature=J4HDCN%2F5ZdrPdPyfE7MjxZBm%2BC0rXFKLVVforWIjKsbEgrETsVXSI91CyPUpf1fzO9QKiC2h1xuX3GPe9%2Fx5oT3bpwzNt04aCRzNE7nfy1nxZtCwRm6TL86G%2FIGVLGIrLaj4NzOqC4Py8ILCtSgV586%2F%2FrW%2BsVrtdqLMUl8c8sWJcY%2BGUvHeN4LQw7qfHi4eAeITjzqKlXghOZoreqDo%2FNtdqXSTlgW9r853crjtlEhbXcClB7BTqR%2B3%2Fg0NaGNKUmcAkCP6%2F7QxxDKYRLA86pR8b6kdI9Cn3MKQrPXkEZfrlDxwLVewl9K1ld1AI3O4KZT6IAE%2FLODhGznXzXyt3w%3D%3D


[{'content': '--- description: Log and visualize data without a W&B account displayed_sidebar: default --- # Anonymous Mode Are you publishing code that you want anyone to be able to run easily? Use Anonymous Mode to let someone run your code, see a W&B dashboard, and visualize results without needing to create a W&B account first. Allow results to be logged in Anonymous Mode with `wandb.init(`**`anonymous="allow"`**`)` :::info **Publishing a paper?** Please [cite W&B](https://docs.wandb.ai/company/academics#bibtex-citation), and if you have questions about how to make your code accessible while using W&B, reach out to us at support@wandb.com. ::: ### How does someone without an account see results? If someone runs your script and you have to set `anonymous="allow"`: 1. **Auto-create temporary account:** W&B checks for an account that\'s already signed in. If there\'s no account, we automatically create a new anonymous account and save that API key for the session. 2. **Log results qui

In [77]:
from scipy.spatial.distance import cdist
from sklearn.feature_extraction.text import TfidfVectorizer

class Retriever:
    def __init__(self):
        self.vectorizer = TfidfVectorizer(stop_words="english", analyzer="char_wb", ngram_range=(1, 2))
        self.index = None
        self.data = None

    def index_data(self, data):
        self.data = data
        docs = [doc['cleaned_content'] for doc in data]
        self.index = self.vectorizer.fit_transform(docs)

    # @weave.op()
    def search(self, query, k=5):
        query_vec = self.vectorizer.transform([query])
        cosine_distances = cdist(query_vec.todense(), self.index.todense(), metric="cosine")[0]
        top_k_indices = cosine_distances.argsort()[:k]
        output = []
        for idx in top_k_indices:
            output.append({
                "source": self.data[idx]["metadata"]["source"],
                "text": self.data[idx]["cleaned_content"],
                "score": 1 - cosine_distances[idx]
            })
        return output


In [95]:
retriever = Retriever()
retriever.index_data(chunked_data)
retriever.search("How do I get get started with wandb?")

[{'source': 'guides/prompts/quickstart.md',
  'text': 'link generated in the previous step. This will redirect you to your Project workspace in the W&B App. Select a run you created to view the trace table, trace timeline and the model architecture of your LLM. ![](/images/prompts/trace_timeline_detailed.png) ### 6. LangChain Context Manager Depending on your use case, you might instead prefer to use a context manager to manage your logging to W&B: ```python from langchain.callbacks import wandb_tracing_enabled # unset the environment variable and use a context manager instead if "LANGCHAIN_WANDB_TRACING" in os.environ: del os.environ["LANGCHAIN_WANDB_TRACING"] # enable tracing using a context manager with wandb_tracing_enabled(): math_agent.run("What is 5 raised to .123243 power?") # this should be traced math_agent.run("What is 2 raised to .123243 power?") # this should not be traced ``` Please report any issues with this LangChain integration to the [wandb repo](https://github.com/w

## Generating a response

In [96]:
import nest_asyncio
nest_asyncio.apply()
import asyncio
import dotenv
dotenv.load_dotenv()

import os

from openai import AsyncOpenAI

client = AsyncOpenAI(api_key=os.environ["OPENAI_API_KEY"])


# @weave.op()
async def generate_response(query):
    context = retriever.search(query)
    context_text = "\n".join([f"Source: {item['source']}\nText: {item['text']}\n\n" for item in context])


    system_message = {
        "role": "system",
        "content": "You are a helpful customer support assistant that can answer questions about W&B\n\n"
        "Your answers must be based only on the provided context.\n\n"
        f"<context>\n{context_text}\n</context>"
    }

    user_message = {
        "role": "user",
        "content": f"Question: {query}\n\nAnswer:"
    }


    response = await client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[system_message, user_message]
    )

    return response.choices[0].message.content


In [98]:
response = asyncio.run(generate_response("How do I get get started with wandb?"))
print(response)

To get started with W&B, you can follow these steps:

1. Import Trace and start a Weights & Biases run using `wandb.init`.
2. Log data to a Trace by querying and logging the results.
3. Use the W&B App to view the trace table, trace timeline, and model architecture of your Language Model (LLM).

Remember to install the necessary version of `wandb` (>= 0.15.4) and log in with your Weights & Biases API key when prompted.


## Evaluating the RAG system

Get from data from the docs website [FAQs](https://docs.wandb.ai/guides/technical-faq) to test the system.

In [136]:
# !pip install -U -qqq ranx

### Collecting Some data for Evals

In [105]:
eval_samples = [
    {"query": "What is the difference between .log() and .summary?", "source": "guides/technical-faq/general.md", },
    {"query": "How do I switch between accounts on the same machine?", "source": "guides/technical-faq/general.md"},
    {"query": "How is W&B different from TensorBoard?", "source": "guides/technical-faq/general.md"},
    {"query": "What is the difference between team and organization?", "source": "guides/technical-faq/admin.md"},
    {"query": "What is the difference between team and entity? As a user - what does entity mean for me?", "source": "guides/technical-faq/admin.md"},
    {"query": "Can I just log metrics, no code or dataset examples?", "source": "guides/technical-faq/metrics-and-performance.md"},
    {"query": "How can I log a metric that doesn't change over time such as a final evaluation accuracy?", "source": "guides/technical-faq/metrics-and-performance.md"},
    {"query": "How many runs to create per project?", "source": "guides/technical-faq/metrics-and-performance"},
    {"query": "Can I run wandb offline?", "source": "guides/technical-faq/setup.md"},
    {"query": "How do I deal with network issues?", "source": "guides/technical-faq/troubleshooting.md"},
    {"query": "What happens if internet connection is lost while I'm training a model?", "source": "guides/technical-faq/troubleshooting.md"},
    {"query": "Where do I find my API key?", "source": "quickstart.md"},
    {"query": "How to create a W&B Experiment", "source": "guides/track/launch.md"},
    {"query": "Log an artifact to W&B", "source": "guides/track/launch.md"},
    {"query": "Log a table to a run", "source": "track/log/log-tables.md"},
    {"query": "How do I log a list of values?", "source": "track/log/logging-faqs.md"},
    {"query": "Is there a way to add extra values to a sweep, or do I need to start a new one?", "source": "guides/sweep/faqs.md"},
    {"query": "Can we flag boolean variables as hyperparameters?", "source": "guides/sweep/faqs.md"},
    {"query": "How do I programmatically access the human-readable run name?", "source": "guides/track/tracking-faq.md"},
    {"query": "How can I save the git commit associated with my run?", "source": "guides/track/tracking-faq.md"}
]
print(len(eval_samples))

20


### Evaluating the Retriever

In [107]:
import numpy as np
import pandas as pd

def calculate_mrr(retriever, eval_samples, k=5):
    results = []
    for sample in eval_samples:
        query = sample['query']
        expected_source = sample['source']
        
        search_results = retriever.search(query, k=k)
        
        # the rank of the expected source
        for rank, result in enumerate(search_results, 1):
            if result['source'] == expected_source:
                mrr_score = 1 / rank
                break
        else:
            # expected source not found in top k results
            mrr_score = 0
        
        results.append({
            'Query': query,
            'Expected Source': expected_source,
            'MRR Score': mrr_score
        })
    
    df = pd.DataFrame(results)
    df['MRR Score'] = df['MRR Score'].astype(float)  # Ensure MRR Score is float
    
    return df

# Evaluate
results_df = calculate_mrr(retriever, eval_samples)
display(results_df)

,Query,Expected Source,MRR Score
0,What is the difference between .log() and .sum...,guides/technical-faq/general.md,0.500000
1,How do I switch between accounts on the same m...,guides/technical-faq/general.md,1.000000
2,How is W&B different from TensorBoard?,guides/technical-faq/general.md,0.333333
3,What is the difference between team and organi...,guides/technical-faq/admin.md,1.000000
4,What is the difference between team and entity...,guides/technical-faq/admin.md,1.000000
5,"Can I just log metrics, no code or dataset exa...",guides/technical-faq/metrics-and-performance.md,1.000000
6,How can I log a metric that doesn't change ove...,guides/technical-faq/metrics-and-performance.md,1.000000
7,How many runs to create per project?,guides/technical-faq/metrics-and-performance,0.000000
8,Can I run wandb offline?,guides/technical-faq/setup.md,1.000000
9,How do I deal with network issues?,guides/technical-faq/troubleshooting.md,0.500000


In [110]:
overall_mrr = results_df['MRR Score'].mean()
print(f"Mean MRR Score: {overall_mrr:.4f}")

# Calculate and print overall MRR score
display(pd.DataFrame(results_df['MRR Score'].describe()).T)

Overall MRR Score: 0.4667


,count,mean,std,min,25%,50%,75%,max
MRR Score,20.0,0.466667,0.473262,0.0,0.0,0.416667,1.0,1.0


#### Evaluating on other metrics

In [137]:
from ranx import Qrels, Run, evaluate
import pandas as pd

METRICS = ["ndcg@5", "map@5", "mrr", "hit_rate", "precision", "recall", "f1"]
def evaluate_retriever(retriever, eval_samples, k=5, metrics=METRICS):
    # Prepare qrels_dict
    qrels_dict = {}
    for i, sample in enumerate(eval_samples):
        qrels_dict[f"q_{i}"] = {sample['source']: 1}  # Assuming relevance of 1 for the correct source

    # Prepare run_dict
    run_dict = {}
    for i, sample in enumerate(eval_samples):
        query = sample['query']
        results = retriever.search(query, k=k)
        run_dict[f"q_{i}"] = {result['source']: result['score'] for result in results}

    # Create Qrels and Run objects
    qrels = Qrels(qrels_dict)
    run = Run(run_dict)

    # Compute metrics
    score_dict = evaluate(qrels, run, metrics, return_mean=False)

    # Combine eval_samples and scores into a DataFrame
    results_df = pd.concat([pd.DataFrame(eval_samples), pd.DataFrame(score_dict)], axis=1)
    
    return results_df

In [138]:
# Usage
results_df = evaluate_retriever(retriever, eval_samples)
display(results_df)

print("\nMean Overall Scores:")
display(pd.DataFrame(results_df[METRICS].mean()).T)

print("\nOverall Score Statistics:")
display(pd.DataFrame(results_df[METRICS].describe()).T)

,query,source,ndcg@5,map@5,mrr,hit_rate,precision,recall,f1
0,What is the difference between .log() and .sum...,guides/technical-faq/general.md,0.630930,0.50,0.50,1.0,0.20,1.0,0.333333
1,How do I switch between accounts on the same m...,guides/technical-faq/general.md,1.000000,1.00,1.00,1.0,0.20,1.0,0.333333
2,How is W&B different from TensorBoard?,guides/technical-faq/general.md,1.000000,1.00,1.00,1.0,0.20,1.0,0.333333
3,What is the difference between team and organi...,guides/technical-faq/admin.md,0.000000,0.00,0.00,0.0,0.00,0.0,0.000000
4,What is the difference between team and entity...,guides/technical-faq/admin.md,1.000000,1.00,1.00,1.0,0.20,1.0,0.333333
5,"Can I just log metrics, no code or dataset exa...",guides/technical-faq/metrics-and-performance.md,0.000000,0.00,0.00,0.0,0.00,0.0,0.000000
6,How can I log a metric that doesn't change ove...,guides/technical-faq/metrics-and-performance.md,0.000000,0.00,0.00,0.0,0.00,0.0,0.000000
7,How many runs to create per project?,guides/technical-faq/metrics-and-performance,0.000000,0.00,0.00,0.0,0.00,0.0,0.000000
8,Can I run wandb offline?,guides/technical-faq/setup.md,0.000000,0.00,0.00,0.0,0.00,0.0,0.000000
9,How do I deal with network issues?,guides/technical-faq/troubleshooting.md,0.000000,0.00,0.00,0.0,0.00,0.0,0.000000



Mean Overall Scores:


,ndcg@5,map@5,mrr,hit_rate,precision,recall,f1
0,0.484627,0.4625,0.4625,0.55,0.115,0.55,0.19



Overall Score Statistics:


,count,mean,std,min,25%,50%,75%,max
ndcg@5,20.0,0.484627,0.474861,0.0,0.0,0.530803,1.000000,1.00
map@5,20.0,0.462500,0.474861,0.0,0.0,0.375000,1.000000,1.00
mrr,20.0,0.462500,0.474861,0.0,0.0,0.375000,1.000000,1.00
hit_rate,20.0,0.550000,0.510418,0.0,0.0,1.000000,1.000000,1.00
precision,20.0,0.115000,0.107728,0.0,0.0,0.200000,0.200000,0.25
recall,20.0,0.550000,0.510418,0.0,0.0,1.000000,1.000000,1.00
f1,20.0,0.190000,0.177408,0.0,0.0,0.333333,0.333333,0.40


### Evaluating the Response